In [1]:
import sys
from web3 import Web3
from misc import wrapper, load_yaml, load_json

In [343]:
from secp256k1 import PrivateKey
from hexbytes import HexBytes

In [62]:
import ecies
import json
import base64

#### LOAD CONFIG

In [2]:
config = load_yaml('./config.yml')

#### CONNECT

In [3]:
def connect():
    
    # CONNECT TO BLOCKCHAIN GATEWAY
    address = config.gateway_uri
    instance = Web3(Web3.WebsocketProvider(config.gateway_uri))
    
    # NOT RESPONDING
    if not instance.isConnected():
        raise Exception('COULD NOT CONNECT TO GATEWAY! ABORTING...')
    
    # OTHEREWISE, RETURN INSTANCE
    return instance

In [449]:
instance = connect()

In [445]:
def serialize_pubkey(keypair):
    
    # SERIALIZE
    hex_bytes = HexBytes(keypair.private)
    raw = PrivateKey(hex_bytes)
    uncompressed = raw.pubkey.serialize(compressed=False)
    
    return uncompressed.hex(), raw.private_key.hex()

In [446]:
def generate_keys():
    
    # GENERATE ETH KEY
    eth_key = ecies.utils.generate_eth_key()
    
    # DERIVE PRIVATE KEY
    private_key = eth_key.to_hex()
    
    # DERIVE PUBLIC KEY
    public_key = eth_key.public_key.to_hex()
    
    return public_key, private_key

In [447]:
def encrypt(message, public_key):
    
    # PROCESS MESSAGE
    stringified = json.dumps(message)
    encoded = stringified.encode()
    
    # ENCRYPT & ENCODE TO BASE64
    encrypted = ecies.encrypt(public_key, encoded)
    converted = base64.b64encode(encrypted)
    
    # RETURN AS STRING
    return converted.decode('utf-8')

In [448]:
def decrypt(message, private_key):
    
    # DECODE INPUT
    encoded = message.encode()
    converted = base64.b64decode(encoded)
    
    # DECRYPT & PARSE
    decrypted = ecies.decrypt(private_key, converted)
    stringified = decrypted.decode('utf-8')
    
    # RETURN AS DICT
    return json.loads(stringified)

In [492]:
pub, priv = serialize_pubkey(config.accounts[0])

In [493]:
payload = { 'foo': 'babr' }

In [494]:
encrypted = encrypt(payload, pub)

In [495]:
result = decrypt(encrypted, priv)

In [496]:
payload == result

True